In [1]:
# This experiment is to decide the max number of features to be used in TFIDf (1,3)

In [ ]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''
os.environ['AWS_DEFAULT_REGION'] = 'ap-southeast-2'

In [16]:
import mlflow
mlflow.set_tracking_uri("http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/")
mlflow.set_experiment("Exp3_TFIDF max feature selection")

<Experiment: artifact_location='s3://sentiment-mlops-bucket-s3/268664869246559694', creation_time=1761258145848, experiment_id='268664869246559694', last_update_time=1761258145848, lifecycle_stage='active', name='Exp3_TFIDF max feature selection', tags={}>

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report
import mlflow.sklearn
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
df = pd.read_csv('cleaned_dataset.csv')

In [19]:
df.head(2)

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1


In [20]:
df.shape

(36661, 2)

In [21]:
# Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 2) # Found from experiment 2

    vectorizer = TfidfVectorizer(ngram_range = ngram_range, max_features = max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], 
                                                        df['category'], 
                                                        test_size = 0.2, 
                                                        random_state = 42,
                                                        stratify = df['category']
                                    )
    
    # Convert the clean_comment column into a vector so we can pass it to the model
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    with mlflow.start_run() as run:
        mlflow.set_tag("mlflow.runName", f"TFIDF_1_3_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        mlflow.set_tag("description", f"Random Forest classifier with TFIDF trigrams 1, 3 and max features = {max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)
        
        n_estimators = 100
        max_depth = 15
        # Log Random Forest Parameters
        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train random forest model
        model = RandomForestClassifier(n_estimators = n_estimators, max_depth= max_depth )
        model.fit(X_train, y_train)

        y_pred_test = model.predict(X_test)
        y_pred_train = model.predict(X_train)

        accuracy_test = accuracy_score(y_test, y_pred_test)
        accuracy_train = accuracy_score(y_train, y_pred_train)

        # Log accuracy
        mlflow.log_metric("accuracy_test", accuracy_test)
        mlflow.log_metric("accuracy_train", accuracy_train)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred_test, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)
        
        # Log Confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred_test)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot = True, fmt = "d", cmap = 'Blues')
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion matrix: TFIDF 1, 3, max_features = {max_features}")
        plt.savefig(f"confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

        # Log model
        mlflow.sklearn.log_model(model, 
                                 name = f"random_forest_classifier_tfidf_trigrams_{max_features}",
                                 registered_model_name=f"reg_random_forest_classifier_tfidf_trigrams_{max_features}")


max_feature_values = [1000, 2000, 5000, 10000]

for max_features in max_feature_values:
    run_experiment_tfidf_max_features(max_features)



2025/10/24 09:49:37 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/24 09:49:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'reg_random_forest_classifier_tfidf_trigrams_1000'.
2025/10/24 09:49:48 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: reg_random_forest_classifier_tfidf_trigrams_1000, version 1
Created version '1' of model 'reg_random_forest_classifier_tfidf_trigrams_1000'.


🏃 View run TFIDF_1_3_max_features_1000 at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/268664869246559694/runs/b90c71ec884d43f5ac2c6ca0ad05e6ac
🧪 View experiment at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/268664869246559694


2025/10/24 09:51:16 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/24 09:51:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'reg_random_forest_classifier_tfidf_trigrams_2000'.
2025/10/24 09:51:20 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: reg_random_forest_classifier_tfidf_trigrams_2000, version 1
Created version '1' of model 'reg_random_forest_classifier_tfidf_trigrams_2000'.


🏃 View run TFIDF_1_3_max_features_2000 at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/268664869246559694/runs/677ad86f2eb94e01900b8ce28cd5e7d8
🧪 View experiment at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/268664869246559694


2025/10/24 09:51:40 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/24 09:51:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'reg_random_forest_classifier_tfidf_trigrams_5000'.
2025/10/24 09:51:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: reg_random_forest_classifier_tfidf_trigrams_5000, version 1
Created version '1' of model 'reg_random_forest_classifier_tfidf_trigrams_5000'.


🏃 View run TFIDF_1_3_max_features_5000 at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/268664869246559694/runs/ac0e48ce57a844dea6f50310f9718dc3
🧪 View experiment at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/268664869246559694


2025/10/24 09:52:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/10/24 09:52:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'reg_random_forest_classifier_tfidf_trigrams_10000'.
2025/10/24 09:52:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: reg_random_forest_classifier_tfidf_trigrams_10000, version 1
Created version '1' of model 'reg_random_forest_classifier_tfidf_trigrams_10000'.


🏃 View run TFIDF_1_3_max_features_10000 at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/268664869246559694/runs/49147ef08c27452bb20790d468112172
🧪 View experiment at: http://ec2-13-211-98-126.ap-southeast-2.compute.amazonaws.com:5000/#/experiments/268664869246559694


In [ ]:
# Chposing max features as 1000 from experiments